## Object Detection using InceptionResNet

#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model

#### Load data

In [3]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,../images/01.xml,79,267,111,156
1,../images/02.xml,508,926,343,503
2,../images/03.xml,68,335,94,152
3,../images/04.xml,113,208,86,110
4,../images/05.xml,40,164,128,192


In [ ]:
# Get file path for a given XML file
def getFilePath(filename):
    # Extract the name of the corresponding image file and create a file path for it
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('../images/', filename)
    # Return the file path for the image file
    return filepath_image

# Get list of image file paths
image_path = list(df['filepath'].apply(getFilePath))
